# Quickstart Guide: OpenAI
Langchain is still changing rapidly. 
See [quickstart](https://python.langchain.com/docs/get_started/quickstart) for up-to-date instructions.

Set OPENAI_API_KEY within .env before running this step:

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

Because langsmith is new, the llm will not give a satisfactory answer without additional context.

In [3]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in several ways:\n\n1. Test Automation: Langsmith can generate test scripts and automate the execution of test cases. It can create a variety of test scenarios with different inputs and expected outputs, saving time and effort in manual testing.\n\n2. Test Data Generation: Langsmith can generate synthetic test data that covers a wide range of inputs, allowing for extensive and comprehensive testing. This helps uncover potential bugs and edge cases that might not be identified with limited real-world data.\n\n3. Test Case Prioritization: Langsmith can analyze the codebase and identify critical areas that require more attention in testing. It can assist in prioritizing test cases based on code complexity, code coverage, and potential impact, ensuring that the most crucial parts of the application are thoroughly tested.\n\n4. Continuous Testing: Langsmith can be integrated into the Continuous Integration/Continuous Delivery (CI/CD) pipeli

^ It is totally BSing and has no idea we are talking about a langchain-related utility.

Using a prompt template...

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

With LCEL piping syntax (new recommended syntax)

In [5]:
chain = prompt | llm
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can be a valuable tool for testing in a variety of ways. Here are some ways in which Langsmith can assist with testing:\n\n1. Test Data Generation: Langsmith can generate realistic and diverse test data automatically. It can create synthetic data that simulates real-world scenarios, helping to uncover edge cases and validate the functionality of your software.\n\n2. Test Case Generation: Langsmith can automatically generate test cases based on your specifications or requirements. It can analyze your codebase and generate test cases that cover different paths and conditions, reducing the manual effort required for test case creation.\n\n3. Test Automation: Langsmith can integrate with popular testing frameworks and tools, allowing you to automate the execution of test cases. This saves time and effort by eliminating the need for manual execution of repetitive tests.\n\n4. Performance Testing: Langsmith can simulate different user loads and stress test your a

Adding an output parser to the end of the pipeline to grab strings instead of objs

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can greatly assist with testing by providing a powerful and versatile language generation capability. Here are a few ways in which Langsmith can help:\n\n1. Test Data Generation: Langsmith can generate a wide range of test data, including realistic names, addresses, dates, phone numbers, and more. This can be particularly useful for populating test databases or creating test scenarios with different data variations.\n\n2. Test Scenario Generation: With Langsmith, you can generate complex and diverse test scenarios by automatically generating natural language descriptions of test cases. This can save significant time and effort in manually creating test scenarios.\n\n3. Automated Test Script Generation: Langsmith's language generation capabilities can be leveraged to automatically generate test scripts in various programming languages. By providing input parameters and expected outputs, Langsmith can generate the code required to execute the test cases.\n\n4. Test Documentati

## Adding retrieval chain
Use langchains tools to add a vectorstore for providing langchain-specific context for langsmith.

Make sure beautifulsoup4 is installed into the notebook environment.

In [7]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Make sure a FAISS lib is installed before next step...

In [9]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

With the vectorstore populated, build the retrieval chain.

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

Could be run by passing directly...

In [11]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

But to get the full context, use a retreival chain to pull from the vectorstore:

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing a way to construct datasets and evaluate the performance of prompts or chains. It allows users to upload datasets and run chains over the data points, logging the results to a new project associated with the dataset. Users can review these results and assign feedback to runs, marking them as correct or incorrect. Additionally, LangSmith provides evaluators that can be specified during a test run to evaluate the results. While these evaluators may not be perfect, they can guide users to examples that should be further examined. Furthermore, LangSmith offers annotation queues for manual review and annotation of runs, allowing users to assess subjective qualities that automatic evaluators struggle with and validate auto-evaluated runs.


Now it is talking about the real langsmith.

## Conversation Retrieval Chain
Adding cross-message memory for chatbots by feeding back the chat history as context.

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Test by asking a follow-up question that assumes awareness of a prior context

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

The objects returned are query results relevant to the question from the vectorstore.

In [15]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [16]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
print(response['answer'])

LangSmith can help test your LLM applications by providing features that simplify the testing process. Here are some ways LangSmith supports testing:

1. Dataset Uploading: LangSmith simplifies dataset uploading, allowing you to easily upload and manage datasets for testing.

2. Running Chains Over Data Points: Once you have a dataset, you can run chains over the data points using the LangSmith client. This allows you to test your prompt or chain by visualizing the outputs. The results can be logged to a new project associated with the dataset for later review.

3. Feedback and Evaluation: LangSmith makes it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app. You can review and evaluate the results, and LangSmith provides aggregate statistics for each test project.

4. Evaluators: LangSmith provides a set of evaluators in the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the resul

## Agent
Let the LLM decide and take action using a list of tools.

In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

Make sure TAVILY_API_KEY is setup in .env before next step...

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

load_dotenv()
search = TavilySearchResults()
tools = [retriever_tool, search]

In [19]:
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the execution of te

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been tested and may require additional test cases.\n\n4. Test Prioritization: LangSmith can 

In [21]:
agent_executor.invoke({"input": "what is the weather in Austin TX?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Austin TX'}`


[{'url': 'https://www.kxan.com/news/local/austin/austin-2024-weather-data/', 'content': 'The chart below shows the high temperature recorded in Austin each day in 2024.  Temperatures, rain, severe storms: Explore Austin’s 2024 weather data by: Christopher Adams  Hourly Forecast Austin-Travis County City continues preparing for ice storms in 2024  Austin’s high temperature in 2023 was 110°, on Aug. 17. The low temperature for the year was 29°, set on both Jan. 3173° #1: 73° (Jan 11) #2: 70° (Jan 8) #3: 67° (Jan 12) #4: 67° (Jan 10) #5: 64° (Jan 7 +1 other) Coldest Low 16° #1: 16° (Jan 17) #2: 16° (Jan 15) #3: 17° (Jan 16) #4: 17° (Jan 14) #5: 32° (Jan 10)...'}, {'url': 'https://en.climate-data.org/north-america/united-states-of-america/texas/austin-714914/t/january-1/', 'content': 'all information about the weather in Austin in January:  Austin weather in January Au

{'input': 'what is the weather in Austin TX?',
 'output': 'I found some information about the weather in Austin, TX. Here are a few sources you can check for more details:\n\n1. [KXAN News](https://www.kxan.com/news/local/austin/austin-2024-weather-data/): This article provides a chart showing the high temperatures recorded in Austin each day in 2024.\n\n2. [Climate Data](https://en.climate-data.org/north-america/united-states-of-america/texas/austin-714914/t/january-1/): This website offers information about the weather in Austin in January, including average temperatures, precipitation, and humidity.\n\n3. [Statesman](https://www.statesman.com/story/news/local/2024/01/24/road-closures-texas-map-txdox-austin-weather/72337375007/): This article discusses road closures in Central Texas due to rain and thunderstorms. It also mentions the weather forecast for the upcoming week, with cool temperatures and no rain.\n\n4. [Statesman](https://www.statesman.com/story/news/local/2024/01/25/rain

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.

2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.

3. Essay Editing: LangSmith can assist with editing and proofreading your application essays, ensuring they are well-written, coherent, and free of grammatical errors. This can enhance the overall quality of your application.

4. Interview Preparation: LangSmith can provide guidance and practice for your LLM interviews. This can include mock interviews, feedback on your responses, and tips on how to effe

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.\n\n2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.\n\n3. Essay Editing: LangSmith can assist with editing and proofreading your application essays, ensuring they are well-written, coherent, and free of grammatical errors. This can enhance the overall quality of your application.\n\n4. Interview Preparation: LangSmith can provide guidance 